In [56]:
import os
import numpy as np
import torch
import torch.nn as nn

In [57]:
import pyro
from pyro.distributions import Normal
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

In [58]:
#for CI testing
smoke_test = ('CI' in os.environ)
pyro.enable_validation(True)

In [59]:
#generate toy dataset
#one feature
#w=3
#b=1

N = 100 #size of toy data
def build_linear_dataset(N , p=1,noise_std =0.01):
    X = np.random.rand(N,p)
    w = 3*np.ones(p)
    #np.repeat(1,N) basically adds b =1 
    y = np.matmul(X,w) + np.repeat(1,N) + np.random.normal(0,noise_std,size=N)
    y=y.reshape(N,1)
    X,y = torch.tensor(X).type(torch.Tensor), torch.tensor(y).type(torch.Tensor)
    data = torch.cat((X,y),1)
    assert data.shape == (N,p+1)
    return data

In [60]:
class RegressionModel(nn.Module):
    def __init__(self ,p):
        #p : number of features
        super(RegressionModel, self).__init__()
        self.linear = nn.Linear(p,1)
    def forward(self, x):
        return self.linear(x)
    
regression_model = RegressionModel(1)

In [61]:
#training
#mean sq err as loss
#adam optimizer
#learning rate 0.01
#iterations 500
loss_fn = torch.nn.MSELoss(reduction='sum')
optim = torch.optim.Adam(regression_model.parameters(),lr=0.5)
num_iterations = 1000 if not smoke_test else 2

In [35]:
def main():
    data = build_linear_dataset(N)
    x_data = data[:,:-1]
    y_data = data[: , -1]
    for j in range(num_iterations):
        #run model forward
        y_pred = regression_model(x_data).squeeze(-1)
        
        #calculate mse loss
        loss = loss_fn(y_pred,y_data)
        
        #initialise gradients to 0
        optim.zero_grad()
        
        #backpropagate
        loss.backward()
        
        #take a gradient step
        optim.step()
        
        if(j+1)%50 == 0:
            print("[iteration %04d] loss : %.4f" %(j+1,loss.item()))
            
            
    print("learned parameters : ")
    for name, param in regression_model.named_parameters():
        print("%s: %.3f" %(name,param.data.numpy()))

if __name__== '__main__':
    main()
            

[iteration 0050] loss : 0.9076
[iteration 0100] loss : 0.0108
[iteration 0150] loss : 0.0098
[iteration 0200] loss : 0.0097
[iteration 0250] loss : 0.0097
[iteration 0300] loss : 0.0097
[iteration 0350] loss : 0.0097
[iteration 0400] loss : 0.0097
[iteration 0450] loss : 0.0097
[iteration 0500] loss : 0.0097
[iteration 0550] loss : 0.0097
[iteration 0600] loss : 0.0097
[iteration 0650] loss : 0.0097
[iteration 0700] loss : 0.0097
[iteration 0750] loss : 0.0097
[iteration 0800] loss : 0.0097
[iteration 0850] loss : 0.0097
[iteration 0900] loss : 0.0097
[iteration 0950] loss : 0.0097
[iteration 1000] loss : 0.0097
learned parameters : 
linear.weight: 2.999
linear.bias: 1.001


Are we confident of above estimates ?
We need Bayesian modelling to reason about the model uncertainty .


Rather than learning point estimates on w and b , we will learn a distribution over values of w and b consistent with the observed data.

### Bayesian Regression

To make linear regression Bayesian , put priors on w and b .
Priors : distributions representing out prior belief about reasonable values for w and b 

random_module() : takes a given nn.Mosule and turns it into distribution over the same module

In this example , a distribution over regressors . Each parameter in the original regression model is sampled from the provided prior .

In [72]:
#model
def model(data):
    #create unit normal priors over the parameters
    loc , scale = torch.zeros(1,1) , 10*torch.ones(1,1)
    bias_loc , bias_scale = torch.zeros(1), 10*torch.ones(1)
    w_prior = Normal(loc, scale).independent(1)
    b_prior = Normal(bias_loc,bias_scale).independent(1)
    priors = {'linear.weight' : w_prior , 'linear.bias' : b_prior}
    
    #lift module parameters to random variables sampled from the priors
    lifted_module = pyro.random_module("module" , regression_model,priors)
    
    #sample a regressor (which also samples w and b) from the prior
    lifted_reg_module = lifted_module()
    with pyro.iarange("map", N) :
        x_data = data[:, :-1]
        y_data = data[:, -1]
        
        #run the regressor forward conditioned on data
        prediction_mean = lifted_reg_module(x_data).squeeze(-1)
        
        #condition on the observed data
        pyro.sample("obs", Normal(prediction_mean,0.1*torch.ones(data.size(0))),obs=y_data)

In order to do inference we’re going to need a guide : a 
parameterized family of distributions over w and b. 

Writing down a guide will proceed in close analogy to the 
construction of our model, 
with the key __difference__ that the 
guide parameters need to be trainable. To do this we register 
the guide parameters in the ParamStore using pyro.param().

In [73]:
softplus = torch.nn.Softplus()
def guide(data):
    #variational parameters
    w_loc = torch.randn(1,1)
    
    #initialise scales to be pretty narrow
    w_log_sig = torch.tensor(-3.0 * torch.ones(1, 1) + 0.05 * torch.randn(1, 1))
    b_loc = torch.randn(1)
    b_log_sig = torch.tensor(-3.0 * torch.ones(1) + 0.05 * torch.randn(1))
    # register learnable params in the param store
    mw_param = pyro.param("guide_mean_weight", w_loc)
    #we use softplus to ensure positivity
    sw_param = softplus(pyro.param("guide_log_scale_weight", w_log_sig))
    mb_param = pyro.param("guide_mean_bias", b_loc)
    sb_param = softplus(pyro.param("guide_log_scale_bias", b_log_sig))
    # guide distributions for w and b
    w_dist = Normal(mw_param, sw_param).independent(1)
    b_dist = Normal(mb_param, sb_param).independent(1)
    dists = {'linear.weight': w_dist, 'linear.bias': b_dist}
    # overload the parameters in the module with random samples
    # from the guide distributions
    lifted_module = pyro.random_module("module", regression_model, dists)
    # sample a regressor (which also samples w and b)
    return lifted_module()

In [74]:
#inference
#use Stochastic Variational Inference (SVI)
#instead of MSE , use ELBO objective

optim = Adam({"lr" : 0.05})
svi = SVI(model, guide, optim, loss=Trace_ELBO())

In [75]:
def main():
    pyro.clear_param_store()
    data = build_linear_dataset(N)
    for j in range(num_iterations):
        #calculate loss and take a gradient step
        loss = svi.step(data)
        if j%100 == 0 :
            print("[iteration %04d] loss: %.4f" %(j+1, loss/float(N)))
            
if __name__ == '__main__':
    main()

[iteration 0001] loss: 704.9314
[iteration 0101] loss: 3.7550
[iteration 0201] loss: 0.2334
[iteration 0301] loss: -0.6487
[iteration 0401] loss: -1.1334
[iteration 0501] loss: -1.0492
[iteration 0601] loss: -1.2557
[iteration 0701] loss: -1.2460
[iteration 0801] loss: -1.2597
[iteration 0901] loss: -1.1881


In [76]:
for name in pyro.get_param_store().get_all_param_names():
    print("[%s]: %.3f" %(name, pyro.param(name).data.numpy()))

[guide_log_scale_bias]: -3.886
[guide_log_scale_weight]: -3.680
[guide_mean_weight]: 3.001
[guide_mean_bias]: 0.989


Evaluate our model by checking its 
predictive accuracy on new test data : point evaluation. 
Sample 20 neural nets from our 
posterior, run them on the test data, then average across 
their predictions and calculate the MSE of the predicted 
values compared to the ground truth.

In [77]:
X = np.linspace(6, 7, num=20)
y = 3 * X + 1
X, y = X.reshape((20, 1)), y.reshape((20, 1))
x_data, y_data = torch.tensor(X).type(torch.Tensor), torch.tensor(y).type(torch.Tensor)
loss = nn.MSELoss()
y_preds = torch.zeros(20, 1)
for i in range(20):
    # guide does not require the data
    sampled_reg_model = guide(None)
    # run the regression model and add prediction to total
    y_preds = y_preds + sampled_reg_model(x_data)
# take the average of the predictions
y_preds = y_preds / 20
print("Loss: ", loss(y_preds, y_data).item())

('Loss: ', 5.852320828125812e-05)
